# What is Profiling

At some point when you're writing big simulations or optimizers, you're going to wonder "Why is my code running so slow?" Can I make it any faster?

For small problems we can do things like looking at data structures and algorithms or estimating Big O of our code, but that is much trickier for complex operations. When working with a language like Python a lot of the modules hide the actual implementation details, so it is hard. It is then much easier to do something like profiling your code.

Profiling is the process of dynamically analyzing the performance of your program in order to identify issues with common parameters such as memory usage or time complexity.

# When to Profile

The number one rule of profiling is "don't do it". That is to say, if you need to profile your code, you have probably developed for too long without doing benchmarks or unit tests. If you get to a point where your code is doing what you want it to do, but it just isn't doing it fast enough, that's the time to break out the profiler and take a look under the hood.

# How to Profile

In python there are several packages that allow you to profile your code.

Here is a [quick guide](https://towardsdatascience.com/how-to-profile-your-code-in-python-e70c834fad89) and here is [another] (https://www.toucantoco.com/en/tech-blog/python-performance-optimization) and here is [yet another](https://dev.to/thefern/profiling-python-with-cprofile-53jf) They're all great resources and I would recommend checking them out.

But we can go a bit more in depth here. 

# Profiling using the CLI
If you're comfortable with command line interface then I would recommend using the python `cProfile` package to do quick analysis of your programs

## Displaying results:
This program displays all of the functions called by your program and shows you how they can be displayed. You are able to sort the data in your choice of format. `tottime` or `cumtime`. Total time is the amount of time spent on this function only. This is great if you want to identify a subroutine that is taking up a lot of time. `cumtime` is the amount of time spent on a single function and all subfunctions

`python -m cProfile -s tottime your_program.py <args>`

## Outputting a file:
Sometimes that's not enough and you want to know what's going on in your stack. That is you want to build a call graph or understand how functions relate to each other. For those more advanced operations, you're going to want to export a binary output that can be analyzed by a profile visualizer.

`python -m cProfile -o output.file your_program.py <args>`

# Profiling using Python

Let's say that you want to do some custom profiling and only want to analyze parts of your code. We can do that with a custom profiling script and what we call a decorator or the @ symbol.

## Line Profiler
Line_profiler is a great tool to use for explanatory profiling, but we will use cProfile since we can do a bit more fun things with it. It is simple to use and gives you lots of information. Check out more info [here](https://medium.com/uncountable-engineering/pythons-line-profiler-32df2b07b290)

## cProfile
This is the standard profiling library in Python.

In [ ]:
import cProfile
import pstats
profiler = cProfile.Profile()  # create profiler object
profiler.enable()  # enable profiler

... # Do stuff here
...
...

profiler.disable()  # disable profiler

# Displaying outputs of Profiling

In python once we have our general code, we are going to want to view it and read it somehow. If you want to interpret your data using csv here is the code example below. We basically want to create a pstats object from our profiler and edit it into a format that looks human readable.

In [ ]:
file_name = f"output"

# Function that makes pstats somewhat readable. 
# You can check out the ones from the tutorials too
result = io.StringIO()
ps = pstats.Stats(profiler, stream=result).sort_stats(sort_key)
ps.print_stats()
result = result.getvalue()
# chop the string into a csv-like buffer
result = "ncalls" + result.split("ncalls")[-1]
result = "\n".join([",".join(line.rstrip().split(None, 5))
                    for line in result.split("\n")])
# save it to disk
with open(f"destination_folder/{file_name}.csv", "w") as write_profile:
    write_profile.write(result)
    write_profile.close()



# Using visualizers

Alternatively, we can use tools built outside of python to accomplish something similar. We just need to dump our profiling file and get the binary.

In [ ]:
# results to binary file
destination = f"destinationfolder/{file_name}.cprof"
profiler.dump_stats(destination)

From there we can either use a callgraph or snake graph to view our outputs.

## Visualizing Profiler

```shell
python3 -m pip install --upgrade pip setuptools wheel
python3 -m pip install snakeviz
python3 -m pip install graphviz
brew install graphviz
```
### SnakeViz

[SnakeViz](https://jiffyclub.github.io/snakeviz/) is a library that creates two types of graphs and a table on an interactive HTML that can be launched in your browser.

With both the Icicle graph and sunburst graph, the cumulative time is represented by the area. You are able to adjust the zoom and the root of the graph by clicking. Double clicking in an empty space sets the root to the parent of the current root if it exists.

The icicle graph uses a rectangular geometry, while sunburst uses a radial geometry for its axis. You are able to close the terminal by pressing `CTRL+C`

#### Icicle Graph

![SnakeViz Icicle Graph example ](https://jiffyclub.github.io/snakeviz/img/icicle.png)

#### Sunburst Graph

![SnakeViz Sunburst Graph Example](https://jiffyclub.github.io/snakeviz/img/sunburst.png)

#### Table

![SnakeViz Table](https://jiffyclub.github.io/snakeviz/img/stats_table.png)

The output also contains an interactive table that lets you search for functions, sort by different keys, and filter the graph.

### GProf2Dot

GProf2dot takes in a binary file, takes the pstats interpretation and outputs it to a graphviz map.

```shell
gprof2dot -f pstats {input_file} | dot -Tpng -o {output_file}
```

![GProf2Dot](https://github.com/jrfonseca/gprof2dot/blob/master/sample.png?raw=true)


### Automating Shell Script
Sometimes, it's hard to know which type of graph is the most useful and it can be difficult to call everything from the command line.

save the below script as `profileviz.sh`

``` 
#!/bin/zsh
FILE=$1
gprof2dot -f pstats "$FILE" | dot -Tpng -o "${FILE%%.*}".png
snakeviz "$FILE"
```

and call it from the terminal with the `*.cprof` file the positional argument. 

#### Call the script from Terminal

```shell
sh profileviz.sh myfile.cprof
```